In [50]:
import os
import cv2
import time
import json
import instaloader
import numpy as np
from pathlib import Path
from urllib import request
from deepface import DeepFace
from dotenv import load_dotenv

In [51]:
def deepface(df, i, deepface_data, deepface_M, deepface_F):
    s = time.perf_counter()
    
    dotenv_path = Path('.env')
    load_dotenv(dotenv_path=dotenv_path)
    
    ig = os.getenv('IG_USERNAME')
    password = os.getenv('IG_PASSWORD')

    L = instaloader.Instaloader()
    L.login(ig, password)
    
    for username in df:
        i += 1
        try:
            p = instaloader.Profile.from_username(L.context, username)
            pic_url = p.profile_pic_url
            resp = request.urlopen(pic_url)
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            demography = DeepFace.analyze(image, actions = ['age', 'gender'])
            deepface_data.append({"Username": username, 
                                  "Gender": demography["gender"], 
                                  "Age": int(demography["age"])})
            print("%d | %.2f | %s |> %s |> %d" % (i, (time.perf_counter() - s), 
                                                  username, demography["gender"], demography["age"]))
            L.close()
        except:
            print("%d | %.2f | %s |> Failed" % (i, (time.perf_counter() - s), username))
            L.close()
             
    for i in range(len(deepface_data)):
        if deepface_data[i]['Gender'] == 'Woman':
            deepface_F += 1
            deepface_data[i]['Gender'] = 'female'
        else:
            deepface_M += 1
            deepface_data[i]['Gender'] = 'male'
            
    age1, age2, age3, age4, age5 = 0, 0, 0, 0, 0
    for i in range(len(deepface_data)):
        if deepface_data[i]['Age'] < 18:
            age1 += 1
        if 18 <= deepface_data[i]['Age']  < 25:
            age2 += 1
        if 25 <= deepface_data[i]['Age']  < 30:
            age3 += 1
        if 30 <= deepface_data[i]['Age']  < 40:
            age4 += 1
        else:
            age5 += 1

    print("\n%d - %d - %d - %d - %d" % (age1, age2, age3, age4, age5))

    deepface_time = time.perf_counter() - s
    print("\nTime: %.2f | Total: %d | Detected: %d | Male: %d | Female: %d\n" 
          % (deepface_time, i, len(deepface_data), deepface_M, deepface_F))

    return deepface_data, deepface_time, deepface_M, deepface_F